<a href="https://colab.research.google.com/github/GiX007/agent-labs/blob/main/03_langchain/11_functional_conversation_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conversational agent

## Setup

In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
dotenv_path = find_dotenv() or '/content/OPENAI_API_KEY.env' # read local .env file
load_dotenv(dotenv_path)

openai_api_key = os.getenv('OPENAI_API_KEY')

## Tools

In [ ]:
!pip install langchain langchain-openai langchain-core langchain-community

In [ ]:
from langchain_core.tools import tool

In [ ]:
import requests
from pydantic import BaseModel, Field
from datetime import datetime, UTC

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""

    BASE_URL = "https://api.open-meteo.com/v1/forecast"

    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.now(UTC)
    time_list = [
        datetime.fromisoformat(time_str.replace("Z", "+00:00")).astimezone(UTC)
    for time_str in results['hourly']['time']
     ]

    temperature_list = results['hourly']['temperature_2m']

    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]

    return f'The current temperature is {current_temperature}°C'

In [ ]:
!pip install wikipedia

In [ ]:
import wikipedia

@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [ ]:
# all tools in a list
tools = [get_current_temperature, search_wikipedia]

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [ ]:
# Convert each @tool function into an OpenAI-compatible function schema
functions = [convert_to_openai_function(f) for f in tools]

# Bind these functions to the model so it can call them automatically
model = ChatOpenAI(temperature=0).bind(functions=functions)

# Build a simple prompt template for system + user messages
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])

# Create a chain: prompt → model (with tool-calls enabled) → parser for tool outputs
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [ ]:
result = chain.invoke({"input": "what is the weather is sf?"})

In [ ]:
# result
result.tool

'get_current_temperature'

In [ ]:
result.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

In [ ]:
from langchain.prompts import MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

```MessagesPlaceholder("agent_scratchpad")``` creates an empty slot in the prompt where the agent can insert its intermediate reasoning, tool call traces, and previous steps. This placeholder is essential for multi-step agent workflows, because it lets the agent “remember” and feed its scratchpad back into each new LLM call.

In [ ]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [ ]:
result1 = chain.invoke({
    "input": "what is the weather is sf?",
    "agent_scratchpad": []
})

In [ ]:
result1

AgentActionMessageLog(tool='get_current_temperature', tool_input={'latitude': 37.7749, 'longitude': -122.4194}, log="\nInvoking: `get_current_temperature` with `{'latitude': 37.7749, 'longitude': -122.4194}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 112, 'total_tokens': 137, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CbfimuaHVcBCBHmHfgnUprhq5HKJh', 'service_tier': 'default', 'finish_reason': 'function_call', 'logprobs': None}, id='run--873e2c53-063b-40ea-91a1-40c23612335c-0', usage_metadata={'input_tokens': 112, 'output_tok

In [ ]:
result1.tool

'get_current_temperature'

In [ ]:
# use tool_input args to get the weather
observation = get_current_temperature.invoke(result1.tool_input)

In [ ]:
observation

'The current temperature is 13.7°C'

In [ ]:
type(result1)

langchain_core.agents.AgentActionMessageLog

In [ ]:
result1.message_log

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 112, 'total_tokens': 137, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CbfimuaHVcBCBHmHfgnUprhq5HKJh', 'service_tier': 'default', 'finish_reason': 'function_call', 'logprobs': None}, id='run--873e2c53-063b-40ea-91a1-40c23612335c-0', usage_metadata={'input_tokens': 112, 'output_tokens': 25, 'total_tokens': 137, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

Previously, we used ```convert_to_openai_function``` to turn a Pydantic model into an OpenAI function schema for defining tools. In contrast, ```format_to_openai_functions``` is used during agent execution to convert the agent's scratchpad (where the agent keeps track of its thoughts, previous steps, and tool calls while figuring out how to answer a question) or intermediate reasoning into a format compatible with OpenAI's function-calling protocol.

In [ ]:
from langchain.agents.format_scratchpad import format_to_openai_functions

In [ ]:
format_to_openai_functions([(result1, observation), ])

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 112, 'total_tokens': 137, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CbfimuaHVcBCBHmHfgnUprhq5HKJh', 'service_tier': 'default', 'finish_reason': 'function_call', 'logprobs': None}, id='run--873e2c53-063b-40ea-91a1-40c23612335c-0', usage_metadata={'input_tokens': 112, 'output_tokens': 25, 'total_tokens': 137, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 FunctionMessage(content='The current temperature is 13.7°C', additional_kwargs={}, res

In [ ]:
result2 = chain.invoke({
    "input": "what is the weather is sf?",
    "agent_scratchpad": format_to_openai_functions([(result1, observation)])
})

In [ ]:
result2

AgentFinish(return_values={'output': 'The current temperature in San Francisco is 13.7°C.'}, log='The current temperature in San Francisco is 13.7°C.')

## Agent

The below function runs an agent in a loop, feeding it the user input and its past reasoning (scratchpad), executing any tools the agent decides to call, collecting the results, and continuing until the agent signals it has finished.

In [ ]:
from langchain.schema.agent import AgentFinish
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = chain.invoke({
            "input": user_input,
            "agent_scratchpad": format_to_openai_functions(intermediate_steps)
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "search_wikipedia": search_wikipedia,
            "get_current_temperature": get_current_temperature,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

Then, we create a new chain that first converts the agent's intermediate steps into the OpenAI function format (scratchpad) and then passes them along to the main chain for execution.

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | chain

**Quick Reminders:**

There are 2 ways to create chains:
  *   ```The prompt | model | ...``` syntax is a **quick, high-level way** to chain prompts, models, and parsers for simple or standard workflows.
  *   ```Runnable``` objects (like ```RunnablePassthrough```) give us more control and flexibility, letting us manipulate inputs, outputs, and intermediate steps explicitly—useful for multi-step agents, custom scratchpads, or streaming pipelines.

In this series of notebooks, we also use 2 ways of interacting with OpenAI models: one with the **client API** (where we used ```gpt-4o-mini```) and one with **ChatOpenAI** (commonly using ```gpt-3.5-turbo```).

In [ ]:
run_agent("what is the weather is sf?")

AgentFinish(return_values={'output': 'The current temperature in San Francisco is 13.7°C.'}, log='The current temperature in San Francisco is 13.7°C.')

In [ ]:
run_agent("what is blockchain?")

AgentFinish(return_values={'output': 'A blockchain is a distributed ledger with growing lists of records (blocks) that are securely linked together via cryptographic hashes. Each block contains a cryptographic hash of the previous block, a timestamp, and transaction data. Blockchains are managed by a peer-to-peer computer network for use as a public distributed ledger, where nodes collectively adhere to a consensus algorithm protocol to add and validate new transaction blocks. The concept of blockchain was introduced by Satoshi Nakamoto in 2008 as the public distributed ledger for bitcoin cryptocurrency transactions. Blockchains are resistant to alteration and are considered secure by design.'}, log='A blockchain is a distributed ledger with growing lists of records (blocks) that are securely linked together via cryptographic hashes. Each block contains a cryptographic hash of the previous block, a timestamp, and transaction data. Blockchains are managed by a peer-to-peer computer netw

In [ ]:
run_agent("hi!")

AgentFinish(return_values={'output': 'Well, hello there! How can I assist you today?'}, log='Well, hello there! How can I assist you today?')

In [ ]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "what is blockchain?"})



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'blockchain'}`


Page: Blockchain
Summary: A blockchain is a distributed ledger with growing lists of records (blocks) that are securely linked together via cryptographic hashes. Each block contains a cryptographic hash of the previous block, a timestamp, and transaction data (generally represented as a Merkle tree, where data nodes are represented by leaves). Since each block contains information about the previous block, they effectively form a chain (compare linked list data structure), with each additional block linking to the ones before it. Consequently, blockchain transactions are resistant to alteration because, once recorded, the data in any given block cannot be changed retroactively without altering all subsequent blocks and obtaining network consensus to accept these changes.
Blockchains are typically managed by a peer-to-peer (P2P) computer network for use as a public distributed ledger, 

{'input': 'what is blockchain?',
 'output': 'A blockchain is a distributed ledger with growing lists of records (blocks) that are securely linked together via cryptographic hashes. Each block contains a cryptographic hash of the previous block, a timestamp, and transaction data. Blockchains are managed by a peer-to-peer computer network for use as a public distributed ledger, where nodes collectively adhere to a consensus algorithm protocol to add and validate new transaction blocks. The concept of blockchain was introduced by Satoshi Nakamoto in 2008 as the public distributed ledger for bitcoin cryptocurrency transactions. Blockchains are resistant to alteration and are considered secure by design. They have inspired various applications and are widely used in cryptocurrencies.'}

In [ ]:
agent_executor.invoke({"input": "my name is bob"})



> Entering new AgentExecutor chain...
Nice to meet you, Bob! How can I assist you today?

> Finished chain.


{'input': 'my name is bob',
 'output': 'Nice to meet you, Bob! How can I assist you today?'}

In [ ]:
agent_executor.invoke({"input": "what is my name"})



> Entering new AgentExecutor chain...
I'm sorry, I don't have access to your personal information. How can I assist you today?

> Finished chain.


{'input': 'what is my name',
 'output': "I'm sorry, I don't have access to your personal information. How can I assist you today?"}

As we see, the agent does not remember, let's fix this.

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [ ]:
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

In [ ]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")

In [ ]:
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [ ]:
# inspect agent_executor object
agent_executor

AgentExecutor(memory=ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='my name is bob', additional_kwargs={}, response_metadata={}), AIMessage(content='Nice to meet you, Bob! How can I assist you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='whats my name', additional_kwargs={}, response_metadata={}), AIMessage(content='You just told me your name is Bob! So, your name is Bob. How can I assist you, Bob?', additional_kwargs={}, response_metadata={}), HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}), AIMessage(content='The current temperature in San Francisco is 13.6°C. Is there anything else you would like to know?', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='my name is George

In [ ]:
agent_executor.invoke({"input": "my name is bob"})



> Entering new AgentExecutor chain...
Nice to meet you, Bob! How can I assist you today?

> Finished chain.


{'input': 'my name is bob',
 'chat_history': [HumanMessage(content='my name is bob', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'output': 'Nice to meet you, Bob! How can I assist you today?'}

In [ ]:
agent_executor.invoke({"input": "whats my name"})



> Entering new AgentExecutor chain...
You just told me your name is Bob! So, your name is Bob. How can I assist you, Bob?

> Finished chain.


{'input': 'whats my name',
 'chat_history': [HumanMessage(content='my name is bob', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='whats my name', additional_kwargs={}, response_metadata={}),
  AIMessage(content='You just told me your name is Bob! So, your name is Bob. How can I assist you, Bob?', additional_kwargs={}, response_metadata={})],
 'output': 'You just told me your name is Bob! So, your name is Bob. How can I assist you, Bob?'}

In [ ]:
agent_executor.invoke({"input": "whats the weather in sf?"})



> Entering new AgentExecutor chain...

Invoking: `get_current_temperature` with `{'latitude': 37.7749, 'longitude': -122.4194}`


The current temperature is 13.6°CThe current temperature in San Francisco is 13.6°C. Is there anything else you would like to know?

> Finished chain.


{'input': 'whats the weather in sf?',
 'chat_history': [HumanMessage(content='my name is bob', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='whats my name', additional_kwargs={}, response_metadata={}),
  AIMessage(content='You just told me your name is Bob! So, your name is Bob. How can I assist you, Bob?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The current temperature in San Francisco is 13.6°C. Is there anything else you would like to know?', additional_kwargs={}, response_metadata={})],
 'output': 'The current temperature in San Francisco is 13.6°C. Is there anything else you would like to know?'}

In this section, we built a simple agent step by step. First, we defined a function to run the agent (```run_agent```) by keeping track of intermediate steps: the model decides which tool to call, we execute the tool, and store the observations. Next, we wrapped this workflow using RunnablePassthrough to create a chain (```agent_chain```) that handles the agent scratchpad and passes it to the model. We then set up a prompt using ChatPromptTemplate with placeholders for chat history and intermediate steps, connected it to the model and an output parser, and finally combined everything into an ```AgentExecutor```. To make the agent conversational, we added ```ConversationBufferMemory``` so it remembers the chat history, and then invoked it with a user input, letting the agent call the appropriate tools and respond intelligently.

## Create a chatbot

Now, let's integrate the agent into a chatbot setup to make the interaction more realistic and conversational.

In [ ]:
@tool
def create_your_own(query: str) -> str:
    """This function can do whatever you would like once you fill it in """
    print(type(query))
    return query[::-1]

In [ ]:
tools = [get_current_temperature, search_wikipedia, create_your_own]

In [ ]:
# import panel as pn  # GUI
# pn.extension()
# import param

# class cbfs(param.Parameterized):

#     def __init__(self, tools, **params):
#         super(cbfs, self).__init__( **params)
#         self.panels = []
#         self.functions = [convert_to_openai_function(f) for f in tools]
#         self.model = ChatOpenAI(temperature=0).bind(functions=self.functions)
#         self.memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
#         self.prompt = ChatPromptTemplate.from_messages([
#             ("system", "You are helpful but sassy assistant"),
#             MessagesPlaceholder(variable_name="chat_history"),
#             ("user", "{input}"),
#             MessagesPlaceholder(variable_name="agent_scratchpad")
#         ])
#         self.chain = RunnablePassthrough.assign(
#             agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
#         ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()
#         self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)

#     def convchain(self, query):
#         if not query:
#             return
#         inp.value = ''
#         result = self.qa.invoke({"input": query})
#         self.answer = result['output']
#         self.panels.extend([
#             pn.Row('User:', pn.pane.Markdown(query, width=450)),
#             pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=450, styles={'background-color': '#F6F6F6'}))
#         ])
#         return pn.WidgetBox(*self.panels, scroll=True)


#     def clr_history(self,count=0):
#         self.chat_history = []
#         return

In [ ]:
# cb = cbfs(tools)

# inp = pn.widgets.TextInput( placeholder='Enter text here…')

# conversation = pn.bind(cb.convchain, inp)

# tab1 = pn.Column(
#     pn.Row(inp),
#     pn.layout.Divider(),
#     pn.panel(conversation,  loading_indicator=True, height=400),
#     pn.layout.Divider(),
# )

# dashboard = pn.Column(
#     pn.Row(pn.pane.Markdown('# QnA_Bot')),
#     pn.Tabs(('Conversation', tab1))
# )
# dashboard

Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Row
                [0] TextInput(placeholder='Enter text here…')
            [1] Divider()
            [2] ParamFunction(function, _pane=Str, defer_load=False, height=400, loading_indicator=True)
            [3] Divider()

In [ ]:
import gradio as gr

def chat(query, history=[]):
    if not query:
        return history
    result = agent_executor.invoke({"input": query})

    # AgentExecutor returns AIMessage objects, extract text
    if hasattr(result, 'get') and 'output' in result:
        answer = result['output']
    elif hasattr(result, 'output_text'):
        answer = result.output_text
    else:
        # fallback: try memory
        answer = str(result)

    history.append((query, answer))
    return history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Enter message...")
    msg.submit(chat, [msg, chatbot], chatbot)

demo.launch()

/tmp/ipython-input-3612071819.py:21: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2643cf8252cfdc4a38.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In this notebook, we first defined tools as we explored analytically in the previous session (notebook), then explicitly created an agent to use them, and finally set up a chatbot powered by this agent to enable real, interactive conversations.